# Trening av RL for å balansere Cartpole

Denne notatboken er en del av [AI for Beginners Curriculum](http://aka.ms/ai-beginners). Den er inspirert av [offisiell PyTorch-veiledning](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) og [denne Cartpole PyTorch-implementasjonen](https://github.com/yc930401/Actor-Critic-pytorch).

I dette eksempelet skal vi bruke RL til å trene en modell til å balansere en stang på en vogn som kan bevege seg til venstre og høyre på en horisontal skala. Vi vil bruke [OpenAI Gym](https://www.gymlibrary.ml/) miljøet for å simulere stangen.

> **Note**: Du kan kjøre koden fra denne leksjonen lokalt (f.eks. fra Visual Studio Code), i så fall vil simuleringen åpnes i et nytt vindu. Når du kjører koden online, kan det være nødvendig å gjøre noen justeringer i koden, som beskrevet [her](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Vi starter med å forsikre oss om at Gym er installert:


In [ ]:
import sys
!{sys.executable} -m pip install gym

La oss opprette CartPole-miljøet og se hvordan vi kan operere i det. Et miljø har følgende egenskaper:

* **Handlingsrom** er settet av mulige handlinger vi kan utføre på hvert steg i simuleringen
* **Observasjonsrom** er rommet av observasjoner vi kan gjøre


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

La oss se hvordan simuleringen fungerer. Følgende løkke kjører simuleringen til `env.step` ikke returnerer avslutningsflagget `done`. Vi vil tilfeldig velge handlinger ved hjelp av `env.action_space.sample()`, noe som betyr at eksperimentet sannsynligvis vil mislykkes veldig raskt (CartPole-miljøet avsluttes når hastigheten til CartPole, dens posisjon eller vinkel er utenfor visse grenser).

> Simuleringen vil åpne i et nytt vindu. Du kan kjøre koden flere ganger og se hvordan den oppfører seg.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Du kan legge merke til at observasjonene inneholder 4 tall. Disse er:  
- Posisjon til vognen  
- Hastighet til vognen  
- Vinkel på stangen  
- Rotasjonshastighet til stangen  

`rew` er belønningen vi mottar for hvert steg. Du kan se at i CartPole-miljøet får du 1 poeng for hvert simuleringssteg, og målet er å maksimere total belønning, altså tiden CartPole klarer å balansere uten å falle.  

Under forsterkende læring er målet vårt å trene en **policy** $\pi$, som for hver tilstand $s$ vil fortelle oss hvilken handling $a$ vi skal utføre, altså $a = \pi(s)$.  

Hvis du ønsker en probabilistisk løsning, kan du tenke på policy som å returnere et sett med sannsynligheter for hver handling, altså $\pi(a|s)$ som betyr sannsynligheten for at vi bør utføre handling $a$ i tilstand $s$.  

## Policy Gradient-metoden  

I den enkleste RL-algoritmen, kalt **Policy Gradient**, vil vi trene et nevralt nettverk til å forutsi neste handling.  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Vi skal trene nettverket ved å kjøre mange eksperimenter og oppdatere nettverket vårt etter hver kjøring. La oss definere en funksjon som vil kjøre eksperimentet og returnere resultatene (såkalt **spor**) - alle tilstander, handlinger (og deres anbefalte sannsynligheter) og belønninger:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Du kan kjøre én episode med et utrent nettverk og observere at total belønning (AKA lengden på episoden) er veldig lav:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

En av de vanskelige aspektene ved policy gradient-algoritmen er å bruke **diskonterte belønninger**. Ideen er at vi beregner vektoren av totale belønninger på hvert steg i spillet, og under denne prosessen diskonterer vi de tidlige belønningene ved hjelp av en koeffisient $gamma$. Vi normaliserer også den resulterende vektoren, fordi vi vil bruke den som vekt for å påvirke treningen vår:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Nå skal vi begynne treningen! Vi skal kjøre 300 episoder, og i hver episode vil vi gjøre følgende:

1. Kjør eksperimentet og samle sporet.
2. Beregn forskjellen (`gradients`) mellom handlingene som ble utført og de predikerte sannsynlighetene. Jo mindre forskjellen er, desto mer sikre er vi på at vi har tatt riktig handling.
3. Beregn diskonterte belønninger og multipliser gradientene med de diskonterte belønningene - dette vil sørge for at steg med høyere belønninger har større innvirkning på sluttresultatet enn de med lavere belønninger.
4. Forventede målhandlinger for vårt nevrale nettverk vil delvis bli hentet fra de predikerte sannsynlighetene under kjøringen, og delvis fra de beregnede gradientene. Vi bruker parameteren `alpha` for å bestemme i hvilken grad gradienter og belønninger tas i betraktning - dette kalles *læringsraten* til forsterkningsalgoritmen.
5. Til slutt trener vi nettverket vårt på tilstander og forventede handlinger, og gjentar prosessen.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Nå la oss kjøre episoden med rendering for å se resultatet:


In [ ]:
_ = run_episode(render=True)

Forhåpentligvis kan du nå se at stangen balanserer ganske bra!

## Actor-Critic Modell

Actor-Critic-modellen er en videreutvikling av policy gradients, der vi bygger et nevralt nettverk for å lære både policy og estimerte belønninger. Nettverket vil ha to utganger (eller du kan se det som to separate nettverk):
* **Actor** vil anbefale handlingen som skal tas ved å gi oss sannsynlighetsfordelingen for tilstanden, som i policy gradient-modellen.
* **Critic** vil estimere hva belønningen vil være fra disse handlingene. Den returnerer totale estimerte belønninger i fremtiden for den gitte tilstanden.

La oss definere en slik modell:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Vi må gjøre noen små endringer i funksjonene `discounted_rewards` og `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Nå skal vi kjøre hovedtreningsløkken. Vi vil bruke en manuell nettverkstreningsprosess ved å beregne riktige tapsfunksjoner og oppdatere nettverksparametere:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Oppsummering

Vi har sett to RL-algoritmer i denne demonstrasjonen: enkel policy gradient og den mer sofistikerte actor-critic. Du kan se at disse algoritmene opererer med abstrakte begreper som tilstand, handling og belønning – derfor kan de brukes i svært forskjellige miljøer.

Forsterkende læring lar oss finne den beste strategien for å løse et problem kun ved å se på den endelige belønningen. Det faktum at vi ikke trenger merkede datasett, gjør at vi kan gjenta simuleringer mange ganger for å optimalisere modellene våre. Likevel er det fortsatt mange utfordringer innen RL, som du kan lære mer om hvis du velger å fordype deg i dette spennende området innen AI.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
